In [1]:
import pandas_datareader.data as data

start_date = '2019-12-01'
end_date = '2019-12-31'

predictors = {
    'sp500': {
      'ticker': '^GSPC',
    },
    'gdax': {
      'ticker': '^GDAXI',
    },
    'nikkei': {
      'ticker': '^N225',
    },
    'gold': {
      'ticker': 'GLD',
    },
    '10y_treasury': {
      'ticker': '^TNX',
    },
}

for predictor in predictors:
    predictors[predictor]['data'] = data.DataReader(predictors[predictor]['ticker'], 'yahoo', start_date, end_date)
    

In [2]:
df = None
drop_cols = ['High','Low','Open','Adj Close','Volume']

for predictor in predictors:
    predictors[predictor]['data_mod'] = predictors[predictor]['data'].rename(columns={"Close": predictor}, errors="raise")
    predictors[predictor]['data_mod'] = predictors[predictor]['data_mod'].drop(drop_cols, axis=1)
    if df is None:
        df = predictors[predictor]['data_mod']
    else:     
        df = df.join(predictors[predictor]['data_mod'], how='outer')

df.tail(10)

,sp500,gdax,nikkei,gold,10y_treasury
Date,,,,,
2019-12-18,3191.139893,13222.160156,23934.429688,139.020004,1.924
2019-12-19,3205.370117,13211.959961,23864.849609,139.380005,1.908
2019-12-20,3221.219971,13318.900391,23816.630859,139.520004,1.917
2019-12-23,3224.010010,13300.980469,23821.109375,139.949997,1.935
2019-12-24,3223.379883,NaN,23830.580078,141.270004,NaN
2019-12-25,NaN,NaN,23782.869141,NaN,NaN
2019-12-26,3239.909912,NaN,23924.919922,142.380005,1.905
2019-12-27,3240.020020,13337.110352,23837.720703,142.330002,1.874
2019-12-30,3221.290039,13249.009766,23656.619141,142.630005,1.895
